In [107]:
import pandas as pd
import gender_guesser.detector as gender
import re

In [108]:
data = pd.read_csv('Data/SIS_bookings_1102.csv')

In [109]:
data

,Unnamed: 0,Day,Facility,Booking Time,Booking Status,Booking Reference Number,Booked for User Name,Booked for User Org Unit,Booked for User Email Address,Use Type,Purpose of Booking
0,0,"11 February 2020, Tuesday",SIS GSR 2-1,19:00-21:00,Confirmed,BK-20200210-000645,WEE XI HWEE,Student,xihwee.wee.2019@sis.smu.edu.sg,Academic,Study for programming
1,1,"11 February 2020, Tuesday",SIS GSR 2-2,11:30-15:30,Confirmed,BK-20200131-000995,"LOW HUI FONG, IRIS",Student,iris.low.2018@sis.smu.edu.sg,Academic,meeting
2,2,"11 February 2020, Tuesday",SIS GSR 2-2,15:30-19:30,Confirmed,BK-20200203-001315,"CHUA ZHI NING, CHARLOTTE",Student,znchua.2019@sis.smu.edu.sg,AdHoc,study
3,3,"11 February 2020, Tuesday",SIS GSR 2-3,12:00-15:00,Confirmed,BK-20200129-001198,LIM JIA WEN,Student,jiawen.lim.2018@business.smu.edu.sg,AdHoc,study
4,4,"11 February 2020, Tuesday",SIS GSR 2-3,15:00-17:00,Confirmed,BK-20200210-001217,"LEK JAN YEE, LOIS",Student,lois.lek.2018@sis.smu.edu.sg,AdHoc,Meeting
...,...,...,...,...,...,...,...,...,...,...,...
444,444,"29 January 2020, Wednesday",SIS GSR 3-5,16:00-20:00,Confirmed,BK-20200116-000861,WONG KA WING,Student,kawing.wong.2018@business.smu.edu.sg,AdHoc,study
445,445,"29 January 2020, Wednesday",SIS GSR 3-5,20:00-22:00,Confirmed,BK-20200129-001128,JOSIAH TAN XIN YI,Student,josiahtan.2019@law.smu.edu.sg,AdHoc,meeting
446,446,"29 January 2020, Wednesday",SIS GSR 3-6,12:00-16:00,Confirmed,BK-20200117-000826,LIN PO CHIEN,Student,pochien.lin.2019@socsc.smu.edu.sg,AdHoc,meeting
447,447,"29 January 2020, Wednesday",SIS GSR 3-6,16:00-18:00,Confirmed,BK-20200127-000439,LIM YIN SHAN,Student,yinshan.lim.2019@sis.smu.edu.sg,Academic,project meeting


In [158]:
# function get gender of the booker
def _get_gender(name):
    chi_sn = ['tan', 'lim', 'lee', 'ng', 'ong', 'wong', 'goh', 'chua', 'chan', 'koh', 'teo', 'ang', 'yeo', 'tay', 'ho', 'low', 'toh', 'sim', 'chong', 'chia']
    d = gender.Detector()
    remove = ', '
    # remove all the non-caps as they are either staff/CCA groups
    if name.isupper() == False:
        return None
    
    # convert entire name to lowercase for easier manipulation
    name = name.lower()
    
    # get gender thru first name if word contains ','
    if ',' in name:
        i = name.index(',')
        word = name[i+1:]
        if word[0] in remove:
            word = word[1:]
        if word[len(word)-1] in remove:
            word = word[:-1]
        word = word.capitalize()
        return d.get_gender(word)
    
    # get gender in common connectors of muslim names
    if 'binte' in name or 'bte' in name or 'bin' in name or 'binti' in name:
        if 'bin' == name:
            return 'male'
        else:
            return 'female'
    
    # get gender by looping thru all parts of the name, removing common surnames (eg. Tan, Lim)
    # get predicted gender of each part of the name
    # eg. Jho Low Xiao Ming => ['unknown', 'unknown', 'andy', 'andy'] (gender_list)
    # finally, get gender by analysing the output-ed gender_list
    else:
        name_list = name.split(' ')
        gender_list = []
        for word in name_list:
            if word[0] in remove:
                word = word[1:]
            if word[len(word)-1] in remove:
                word = word[:-1]
            if word in chi_sn:
                continue
            word = word.capitalize()
            gender_list.append(d.get_gender(word))
            
        # get gender by first part of the name or last part of the name
        # eg. Joe Low Xiao Ming or Low Xiao Ming Joe
        if gender_list[0] == 'male' and gender_list[-1] != 'female':
            return 'male'
        if gender_list[0] == 'female' and gender_list[-1] != 'male':
            return 'female'
        if gender_list[-1] == 'male' and gender_list[0] != 'female':
            return 'male'
        if gender_list[-1] == 'female' and gender_list[0] != 'male':
            return 'female'
        
        # get gender by appending predicted gender of all parts of the name into a list, then getting the mode of the list
        else:
            temp = []
            for gender_pred in gender_list:
                match = re.search('male|female|andy|unknown', gender_pred)
                temp.append(match.group())
            return max(set(temp), key=temp.count) # returns the max 

In [159]:
data['gender_list'] = data['Booked for User Name'].apply(lambda x: test(x))

In [160]:
data.head()

,Unnamed: 0,Day,Facility,Booking Time,Booking Status,Booking Reference Number,Booked for User Name,Booked for User Org Unit,Booked for User Email Address,Use Type,Purpose of Booking,gender_list
0,0,"11 February 2020, Tuesday",SIS GSR 2-1,19:00-21:00,Confirmed,BK-20200210-000645,WEE XI HWEE,Student,xihwee.wee.2019@sis.smu.edu.sg,Academic,Study for programming,unknown
1,1,"11 February 2020, Tuesday",SIS GSR 2-2,11:30-15:30,Confirmed,BK-20200131-000995,"LOW HUI FONG, IRIS",Student,iris.low.2018@sis.smu.edu.sg,Academic,meeting,female
2,2,"11 February 2020, Tuesday",SIS GSR 2-2,15:30-19:30,Confirmed,BK-20200203-001315,"CHUA ZHI NING, CHARLOTTE",Student,znchua.2019@sis.smu.edu.sg,AdHoc,study,female
3,3,"11 February 2020, Tuesday",SIS GSR 2-3,12:00-15:00,Confirmed,BK-20200129-001198,LIM JIA WEN,Student,jiawen.lim.2018@business.smu.edu.sg,AdHoc,study,andy
4,4,"11 February 2020, Tuesday",SIS GSR 2-3,15:00-17:00,Confirmed,BK-20200210-001217,"LEK JAN YEE, LOIS",Student,lois.lek.2018@sis.smu.edu.sg,AdHoc,Meeting,female


In [163]:
data.to_csv('Data/y4nrui.csv')

In [154]:
print(test('JOE LOW HUI FONG'))

male


In [123]:
lst = ['m', 'f', 'f', 'f']
max(set(lst), key=lst.count)

'f'

In [122]:
match = re.search('male|female|andy|unknown', 'mostly_female_male_female')
if match:
    print(match.group())

female
